In [24]:
from scipy.stats import shapiro as stats_shapiro, kruskal as stats_kruskal
from sklearn.metrics import f1_score, accuracy_score, classification_report

In [43]:
import scikit_posthocs

In [2]:
models = ['juliana-ab', 'juliana-svm', 'juliana-dt', 'juliana-rf', 'juliana-gb', 'juliana-knn', 'juliana-mlp']

In [19]:
def report_folder_n(label, fold_no=10):
    f1_scores = []
    accuracies = []
    all_predictions = []
    all_labels = []
    
    for n in range(15):
        predictions = []
        labels = []
        for i in range(1, fold_no + 1):
            with open('results/' + label + '/' + str(n) + '/' + str(i) + '-predicted.txt', 'r') as file:
                for _predictions in file.readlines():
                    predictions += list(_predictions)[:-1]
                    
            with open('results/' + label + '/' + str(n) + '/' + str(i) + '-label.txt', 'r') as file:
                for _labels in file.readlines():
                    labels += list(_labels)[:-1]
                    
        all_predictions += predictions            
        all_labels += labels           
        f1_scores.append(f1_score(labels, predictions, average='weighted'))
        accuracies.append(accuracy_score(labels, predictions))
    
    #print(classification_report(all_labels, all_predictions))
    return f1_scores, accuracies

In [20]:
def shapiro(model):
    print("====> Shapiro-Wilk for " + model)
    f1_scores, accuracies = report_folder_n(model)
    p_value_f1 = stats_shapiro(f1_scores)[1]
    p_value_accuracy = stats_shapiro(accuracies)[1]
    
    print("    p-value F1-score: " + str(p_value_f1), end=" ")
    if p_value_f1 < 0.05:
        print("not gaussian")
    else:
        print("gaussian")
        
    print("    p-value Accuracy: " + str(p_value_accuracy), end=" ")
    if p_value_accuracy < 0.05:
        print("not gaussian\n")
    else:
        print("gaussian\n")

In [21]:
list(map(shapiro, models))

====> Shapiro-Wilk for juliana-ab
    p-value F1-score: 0.18772399425506592 gaussian
    p-value Accuracy: 0.30482861399650574 gaussian

====> Shapiro-Wilk for juliana-svm
    p-value F1-score: 0.32283326983451843 gaussian
    p-value Accuracy: 0.5329688191413879 gaussian

====> Shapiro-Wilk for juliana-dt
    p-value F1-score: 0.5072422623634338 gaussian
    p-value Accuracy: 0.400658518075943 gaussian

====> Shapiro-Wilk for juliana-rf
    p-value F1-score: 0.7010366320610046 gaussian
    p-value Accuracy: 0.7572941780090332 gaussian

====> Shapiro-Wilk for juliana-gb
    p-value F1-score: 1.904447458400682e-06 not gaussian
    p-value Accuracy: 1.4564349157808465e-06 not gaussian

====> Shapiro-Wilk for juliana-knn
    p-value F1-score: 0.343470960855484 gaussian
    p-value Accuracy: 0.8755550384521484 gaussian

====> Shapiro-Wilk for juliana-mlp
    p-value F1-score: 0.8411683440208435 gaussian
    p-value Accuracy: 0.8745509386062622 gaussian



[None, None, None, None, None, None, None]

In [46]:
results = list(map(report_folder_n, models))
f1_scores = list(map(lambda x: x[0], results))
accuracies = list(map(lambda x: x[1], results))
f1_kruskal = stats_kruskal(*f1_scores)
acc_kruskal = stats_kruskal(*accuracies)
print(f1_kruskal, acc_kruskal)

KruskalResult(statistic=99.37308176100635, pvalue=3.3910468756000684e-19) KruskalResult(statistic=99.23905480245028, pvalue=3.6165018378640245e-19)


In [65]:
f1_df = scikit_posthocs.posthoc_dunn(f1_scores, p_adjust = 'sidak').round(4)
print(f1_df.style.to_latex())

\begin{tabular}{lrrrrrrr}
 & 1 & 2 & 3 & 4 & 5 & 6 & 7 \\
1 & 1.000000 & 0.983400 & 0.136800 & 0.000000 & 0.000000 & 0.001100 & 0.000000 \\
2 & 0.983400 & 1.000000 & 0.983400 & 0.000000 & 0.000000 & 0.136800 & 0.000700 \\
3 & 0.136800 & 0.983400 & 1.000000 & 0.000100 & 0.000000 & 0.983400 & 0.105800 \\
4 & 0.000000 & 0.000000 & 0.000100 & 1.000000 & 1.000000 & 0.021600 & 0.760000 \\
5 & 0.000000 & 0.000000 & 0.000000 & 1.000000 & 1.000000 & 0.015400 & 0.680100 \\
6 & 0.001100 & 0.136800 & 0.983400 & 0.021600 & 0.015400 & 1.000000 & 0.967200 \\
7 & 0.000000 & 0.000700 & 0.105800 & 0.760000 & 0.680100 & 0.967200 & 1.000000 \\
\end{tabular}



In [68]:
acc_df = scikit_posthocs.posthoc_dunn(accuracies, p_adjust = 'sidak').round(4)
print(acc_df.style.to_latex())

\begin{tabular}{lrrrrrrr}
 & 1 & 2 & 3 & 4 & 5 & 6 & 7 \\
1 & 1.000000 & 0.980800 & 0.139100 & 0.000000 & 0.000000 & 0.001100 & 0.000000 \\
2 & 0.980800 & 1.000000 & 0.986400 & 0.000000 & 0.000000 & 0.148600 & 0.000800 \\
3 & 0.139100 & 0.986400 & 1.000000 & 0.000100 & 0.000000 & 0.984200 & 0.103900 \\
4 & 0.000000 & 0.000000 & 0.000100 & 1.000000 & 1.000000 & 0.023500 & 0.787700 \\
5 & 0.000000 & 0.000000 & 0.000000 & 1.000000 & 1.000000 & 0.012900 & 0.648500 \\
6 & 0.001100 & 0.148600 & 0.984200 & 0.023500 & 0.012900 & 1.000000 & 0.964400 \\
7 & 0.000000 & 0.000800 & 0.103900 & 0.787700 & 0.648500 & 0.964400 & 1.000000 \\
\end{tabular}

